In [11]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d sainikhileshreddy/food-recognition-2022

100% 4.90G/4.91G [00:58<00:00, 106MB/s]
100% 4.91G/4.91G [00:58<00:00, 89.5MB/s]


In [4]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/food-recognition-2022.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [5]:
# Change the path to the directory where your training data is stored
data_dir = '/content/raw_data/public_training_set_release_2.0'

# Split your data into training and validation sets
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        data_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

Found 39962 images belonging to 1 classes.
Found 39962 images belonging to 1 classes.


In [6]:
# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
# Train the model
model.fit(
        train_generator,
        steps_per_epoch=8000 // 32,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=2000 // 32)

# Save the trained model
model.save('food_recognition_model.h5')

Epoch 1/10
250/250 [==============================] - 307s 1s/step - loss: 0.0027 - accuracy: 0.9987 - val_loss: 2.8959e-07 - val_accuracy: 1.0000
Epoch 2/10
250/250 [==============================] - 301s 1s/step - loss: 5.1443e-07 - accuracy: 1.0000 - val_loss: 1.2288e-08 - val_accuracy: 1.0000
Epoch 3/10
250/250 [==============================] - 305s 1s/step - loss: 1.5136e-07 - accuracy: 1.0000 - val_loss: 2.8175e-09 - val_accuracy: 1.0000
Epoch 4/10
250/250 [==============================] - 290s 1s/step - loss: 3.1740e-06 - accuracy: 1.0000 - val_loss: 3.0120e-10 - val_accuracy: 1.0000
Epoch 5/10
250/250 [==============================] - 297s 1s/step - loss: 7.7718e-08 - accuracy: 1.0000 - val_loss: 1.2894e-10 - val_accuracy: 1.0000
Epoch 6/10
250/250 [==============================] - 316s 1s/step - loss: 9.7006e-08 - accuracy: 1.0000 - val_loss: 2.1375e-10 - val_accuracy: 1.0000
Epoch 7/10
250/250 [==============================] - 321s 1s/step - loss: 1.0827e-07 - accuracy: 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Load the trained model
model = load_model('food_recognition_model.h5')

# Define the function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    return img_array

# Make a prediction for a given image
def predict_food(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    return predicted_class

# Use the function to make a prediction
predicted_class = predict_food('/content/grain.jpg')
print(f'Predicted food class: {predicted_class}')

1/1 [==============================] - 0s 92ms/step
Predicted food class: 0
